In [ ]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["data/Crayon Kvalitetssystem 02 Personalhåndbok.pdf"]
).load_data()

In [ ]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
from llama_index.node_parser import SentenceWindowNodeParser

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
)

In [ ]:
from llama_index.node_parser import SemanticSplitterNodeParser

# create the sentence window node parser w/ default settings
node_parser = SemanticSplitterNodeParser.from_defaults(
    buffer_size=10,
    embed_model=embed_model,
)

In [ ]:
from utils import build_index, get_llm_model
from config import settings
llm=get_llm_model(settings, "an assistant")
index=build_index(document, node_parser, llm=llm, save_dir="semantic_splitter_index")

In [ ]:
# Build different indexes

In [ ]:
import os
from config import settings
os.environ["AZURE_OPENAI_API_KEY"]=settings.AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"]=settings.AZURE_OPENAI_API_ENDPOINT
os.environ["OPENAI_API_VERSION"]=settings.AZURE_OPENAI_API_VERSION

In [ ]:
from utils import get_sentence_window_query_engine
from utils_trulens import get_prebuilt_trulens_recorder

In [ ]:
semanticparser_query_engine = get_sentence_window_query_engine(
    index
)

recorder_1 = get_prebuilt_trulens_recorder(
    semanticparser_query_engine,
    app_id='Semantic parser 10'
)

In [ ]:
eval_questions = []
with open('generated_questions.text', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [ ]:
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)

In [ ]:
run_evals(eval_questions, recorder_1, semanticparser_query_engine)

In [ ]:
Tru().run_dashboard()